In [ ]:
%%capture install
try:
  import imlms
  print('Already installed')
except:
  %pip install git+https://github.com/Mads-PeterVC/imlms

In [ ]:
print(install.stdout.splitlines()[-1])

In [ ]:
import torch
import matplotlib.pyplot as plt

# Pytorch

Pytorch is a package that is very commonly used for machine learning. One primary 
reason for this is *automatic differentiation*.

As you probably know analytically differentiating a function is a tiresome process and 
implementing a program to calculate them is generally more complicated than calculating 
the function itself. 

Auto differentiation is a way of avoiding this hurdle. 

### Example: Auto differentiation of a simple function.

We can try auto-diff for the trivial function 

$$
f(x) = x^2
$$

Where the derivative is 

$$
f'(x) = 2x
$$

In [ ]:
def f(x):
    return x**2

def f_prime(x):
    return 2*x

Nothing special about these function definitions, they are completely normal Python code.

In [ ]:
x = torch.linspace(0, 10, 100, requires_grad=True) # Line 1
y = f(x) # Line 2
y_prime_auto = torch.autograd.grad(y, x, grad_outputs=torch.ones_like(y))[0] # Line 3

Here some magic happens that we will digest later, but first visually that the 
two ways of calculating the derivative worked. 

In [ ]:
fig, ax = plt.subplots(figsize=(5, 3))

y_prime = f_prime(x) # Calculate the analytical derivative to compare against. 

ax.plot(x.detach().numpy(), y_prime.detach().numpy(), label="f'(x)")
ax.plot(x.detach().numpy(), y_prime_auto.detach().numpy(), '--', label="f'(x) auto")

difference = torch.abs(y_prime - y_prime_auto).sum()
ax.set_title(f"Sum of differences: {difference.item():.8f}")

ax.legend()
plt.show()

Now lets work through the details of the calculation

1. `x = torch.linspace(0, 10, 100, requires_grad=True)`
    - Here we create a `torch.Tensor` of values ranging from 1 to 100. The `requires_grad=True` argument tells torch that we will want gradients wrt. `x` of any calculations done with `x`. 
2. `y = f(x)`
    - We calculate the function value, nothing special here except that because `x` is a `torch.Tensor` the calculation happens in torch. 
3. `y_prime_auto = torch.autograd.grad(y, x, grad_outputs=torch.ones_like(y))[0]`
    - The `torch.autograd.grad` function will compute the derivative of the first argument wrt. to the second. The `grad_outputs=torch.ones_like(y)` argument is only necessary because we have vectors for `y` rather than just a single scalar. 

You may have noticed that sometimes syntax like `x.detach().numpy()` is used. 

This is done in order for torch to allow the tensor to be converted to a Numpy array. 
The detach statement is telling Torch that it doesn't need to keep track off computations with the 
returned tensor. 

The same can be realized with `with torch.no_grad()` telling torch that computations done within that context 
won't need to have gradients calculated. 

### Exercise: Another polynomial

Consider the function

$$
f(x) = x^3 + x^2 + x
$$
Use torch to compute the derivative in the interval [0, 10] and compare to the analytical solution

In [ ]:
def f(x):
    pass # Your code here

def f_prime(x):
    pass # Your code here

In [ ]:
x = 0 # Replace with your code
y = 0 # Replace with your code
y_prime_auto = 0 # Replace with your code

In [ ]:
fig, ax = plt.subplots(figsize=(5, 3))

y_prime = f_prime(x) # Calculate the analytical derivative to compare against. 

ax.plot(x.detach().numpy(), y_prime.detach().numpy(), label="f'(x)")
ax.plot(x.detach().numpy(), y_prime_auto.detach().numpy(), '--', label="f'(x) auto")

difference = torch.abs(y_prime - y_prime_auto).sum()
ax.set_title(f"Sum of differences: {difference.item():.8f}")

ax.legend()
plt.show()

### Example: Higher-order derivatives

We are not limited to just first-order derivatives, we can compute derivatives to arbitrary order using 
autodifferentiation. 

In [ ]:
def f(x):
    return x**2

In [ ]:
x = torch.linspace(0, 10, 100, requires_grad=True)
y = f(x)
y_prime_auto = torch.autograd.grad(y, x, grad_outputs=torch.ones_like(y), create_graph=True)[0]
y_prime_prime_auto = torch.autograd.grad(y_prime_auto, x, grad_outputs=torch.ones_like(y_prime_auto))[0]

print(y_prime_prime_auto) # This will print a tensor of 2's as expected.

### Example: Multi-dimensional function

We can use autodifferentiation to get all the partial derivatives of a function of 
multiple variables, e.g

$$
f(x, y) = 3 x^2 + 7 y^2 + xy
$$

In [ ]:
def f(tensor):
    x = tensor[0]
    y = tensor[1]
    return 3 * x**2 + 7 * y**2 + x * y

xy = torch.tensor([1.0, 2.0], requires_grad=True)

z = f(xy)
df = torch.autograd.grad(z, xy, create_graph=True)[0]
print(df)

### Exercise: Electric Field

Consider the scalar potential of an electric charge 

$$
V(\bar{r}) = \frac{kq}{|\bar{r}-\bar{r_0}|}
$$

Where $r_0$ is the position of the charge. In two dimensions we could write this as

$$
V(x, y) = \frac{kq}{\sqrt{(x-x_0)^2 + (y-y_0)^2}}
$$

As we know, the electric field is the negative gradient of the potential 
$$
\mathbf{E} = -\nabla V
$$
Use Pytorch to calculate the electric field.

Start by defining a function that calculates the potential as a function of a tensor $X$
where the first element is $x$ and the second is $y$.

In [ ]:
x0 = 5.0
y0 = 5.0

def V(X):
    x = X[0]
    y = X[1]
    # Your code here

Now define another function that calculates the electric field using automatic differentiation

In [ ]:
def E(X):
    # Your code here
    e = 0 
    return e

Then we can try the function

In [ ]:
X = torch.tensor([1.0, 1.0], requires_grad=True)
e = E(X)
print(e)

We can plot the field using a quiver plot

In [ ]:
N = 40
field = torch.zeros((N, N, 2))

x = torch.linspace(0, 10, N)
y = torch.linspace(0, 10, N)

for i, _x in enumerate(x):
    for j, _y in enumerate(y):

        if (_x - x0)**2 + (_y - y0)**2 < 1.0: # This is just a hack to avoid the singularity at (x0, y0).
            continue

        X_tensor = torch.tensor([_x, _y], requires_grad=True)
        field[i, j] = E(X_tensor)

fig, ax = plt.subplots(figsize=(4, 4))

X, Y = torch.meshgrid(torch.linspace(0, 10, N), torch.linspace(0, 10, N))

ax.quiver(X, Y, field[:, :, 0].detach().numpy(), 
          field[:, :, 1].detach().numpy(), 
          )

ax.axis('equal')

Which matches the expected field rather well. 

An extension of this exercise would be to have the potential be the result of multiple 
point-charges at different locations - so try that at some point if you like. 

### Example: Gradient Descent & Learning

We have seen a few examples of gradients so far, another is gradient descent. 
With gradient descent we optimize a function by taking small steps in the direction 
of the gradient.

There are many use cases of such optimizations, we can focus on one involving what one 
may call 'fitting', 'regression' or 'learning'. Lets say we have some experimental observations 
$[(x_1, y_1), .., (x_N, y_N)]$ that we wish to explain with a model $M_\theta$ where $\theta$ denotes the parameters of the model

We want to find the parameters of our model that makes it fit the data the best, 
and to do so we want a function that we can minimize wrt. the parameters. One 
intuitive choice is 

$$
L(\theta) = \frac{1}{N}\sum_i^N \Big(M_\theta(x_i) - y_i\Big)^2,
$$

which is the *mean squared error loss* between the model and the observations. 

We are free to choose our model, for this example we will choose a linear model

$$
M_\theta(x) = \alpha x + \beta
$$

So our model has two parameters thus $\theta = [\alpha, \beta]$. 
We want to minimize the loss $L$ wrt. the parameters $\theta$ by iteratively 
taking small steps in the direction of the gradient of $L$ wrt. $\theta$. 
That is, 

$$
\theta_{i+1} = \theta_i - \alpha \nabla_\theta L
$$

Where $\alpha$ is typically called the learning rate. We want to calculate $ \nabla_\theta L$ using automatic differentiation.

Lets set this up, first we could define the model

In [ ]:
def model(x, theta):
    return theta[0] * x + theta[1]

def loss_fn(y_pred, y):
    return torch.mean((y_pred - y)**2)

The next cell creates the data we will fit to

In [ ]:
from imlms.torch import get_example_data

X, y = get_example_data()

And finally we are ready to implement the gradient descent loop

In [ ]:
theta = torch.tensor([1.0, 1.0], requires_grad=True) # Initial parameters for the model.

y_pred_initial = model(X, theta)

N_steps = 100
learning_rate = 0.05

for i in range(N_steps):

    y_pred = model(X, theta)
    loss = loss_fn(y_pred, y)

    if i % 10 == 0:
        print(f"Loss {i}: {loss.item():0.3f}")

    gradient = torch.autograd.grad(loss, theta)[0]

    with torch.no_grad():
        theta -= learning_rate * gradient

y_pred_final = model(X, theta)

In [ ]:
fig, ax = plt.subplots(figsize=(4, 4))

with torch.no_grad():
    ax.plot(X.detach().numpy(), y_pred_initial.detach().numpy(), label="Initial")
    ax.plot(X.detach().numpy(), y_pred_final.detach().numpy(), label="Final")
    ax.scatter(X.detach().numpy(), y.detach().numpy(), label="Data")

ax.legend()

### Exercise: Damped Harmonic Oscillator

After countless hours in the lab you have gathered data on the very important 
phenonenom of quark anti-space mega amplitude. You believe that the property 
you have measured should behave like a damped harmonic oscillator following the 
expression 

$$
M(t) = A \exp^{-\gamma t} \cos(\omega t)
$$

Where $t$ is the time and $A$, $\gamma$ and $\omega$ are parameters that you wish to 
find.

Use the same methodology as above to find the best set of parameters $\theta = [A, \gamma, \omega]$. 

The cell below generates your dataset

In [ ]:
from imlms.torch import get_damped_osc_dataset

t, y = get_damped_osc_dataset()

Define your model and loss function

In [ ]:
def model(t, theta):
    A, gamma, omega = theta
    return 0 # Your code here

def loss_fn(y_pred, y):
    return 0 # Your code here

Then create the training loop 

In [ ]:
theta = torch.tensor([1.0, 1.0, 1.0], requires_grad=True) # Initial parameters for the model.

y_pred_initial = model(t, theta)

N_steps = 200 # You can try other values
learning_rate = 0.05 # You can try other values

for i in range(N_steps):

    # Your code here

    with torch.no_grad():
        theta -= 0 # Your code here

y_pred_final = model(t, theta)

In [ ]:
fig, ax = plt.subplots(figsize=(4, 4))

with torch.no_grad():
    ax.plot(t.detach().numpy(), y_pred_initial.detach().numpy(), label="Initial")
    ax.plot(t.detach().numpy(), y_pred_final.detach().numpy(), label="Final")
    ax.scatter(t.detach().numpy(), y.detach().numpy(), label="Data")

ax.legend()

### Example: Torch OOP

So far our models have been very simple, containing only a few parameters so we have 
managed to just keep them in a tensor `theta`. However, models can have a huge amount of parameters - some language models, such as ChatGPT, are thought to 
have over a trillion parameters. Keep tracking of that many parameters in the way we have been doing would be difficult. 

Our rescue is `torch.nn.Module`, which helps us in several ways. Below is an example, 
where I have implemented the model from the previous exercise as a `torch.nn.Module`-derived 
class.

In [ ]:
class DampenedOscillator(torch.nn.Module):

    def __init__(self):
        super().__init__()
        self.A = torch.nn.Parameter(torch.tensor(1.0))        
        self.omega = torch.nn.Parameter(torch.tensor(1.0))
        self.gamma = torch.nn.Parameter(torch.tensor(1.0))

    def forward(self, t):
        return self.A * torch.exp(-self.gamma * t) * torch.cos(self.omega * t)


Now all the parameters are accesible to us through methods of the our class

In [ ]:
oscillator = DampenedOscillator()

print('Parameters:')
for parameter in oscillator.parameters():
    print(parameter)

print('Named parameters:')
for name, parameter in oscillator.named_parameters():
    print(name, parameter)

Using the model is simple

In [ ]:
t = torch.tensor([1.0])
y = oscillator(t) # Equivalent to calling oscillator.forward(t)
print(t, y)